# MM1 Queuing System

A MM1 queuing system will be used to illustrate the three main simulation methodologies:
- time-stepping
- discrete-events processing
- process-driven simulation

## Using packages


In [ ]:
using SimJulia
using Distributions
using Plots
using StatsPlots
using HypothesisTests

## MM1 Queuing System

- The queuing system has 1 server.
- The interarrival times between clients (packets, ...) are exponential distributed with rate $\lambda$.
- The service times are also exponential distributed with rate $\mu$.

In [ ]:
const λ = 1.0
const μ = 2.0;

### Time-stepping

- A small value for the time increment $\Delta t$ is chosen and every tick of the clock a function that mimics our queuing system is run.
- Exponential distributions can be easily simulated; $P(\text{"arrival"})=\lambda\Delta t$ and $P(\text{"departure"})=\mu\Delta t$.

In [ ]:
Δt = 0.1

In [ ]:
function time_step(nr_in_system::Int)
    if nr_in_system > 0
        if rand() < μ*Δt
            nr_in_system -= 1
        end
    end
    if rand() < λ*Δt
        nr_in_system += 1
    end
    nr_in_system
end

In [ ]:
output = Int[]
push!(output, 0)
t = 0.0
while t < 10
    t += Δt
    result = time_step(output[end])
    push!(output, result)
end
println(output)

- Very easy to implement for simple queuing systems but this become cumbersome if the system gets more complex (number of queues, interactions, other distributions, ...).

- A simulation is always something dynamic, i.e. time is an important feature:
    - When are we in steady-state?
    - How many samples of the system in steady-state are needed, to produce a useful average?
    - How many runs do a need to have some statistics about the variation around the average?
    - These questions are trivial for our example but in real-world applications they are though to answer.
    - A stable single server queue is reset every time the queue becomes idle, so we don't have to worry about steady-state and we can use eg. 1000 time-steps and 100 runs.

### Discrete-event processing

- Looking at the output of the time-stepping procedure, we can observe that a lot of the time-steps the state of our system, i.e. the number of clients in the system does not change. So the procedure does a lot of processing for nothing.
- We can predict the next arrival of a client by sampling an exponential distribution with parameter $\frac{1}{\lambda}$, so can we predict the service time of a client by sampling an exponential distribution with parameter $\frac{1}{\mu}$.
- Only during an arrival of a client or an end of service of a client, the state changes.

In [ ]:
interarrival_distribution = Exponential(1/λ)
service_distribution = Exponential(1/μ)

In [ ]:
function arrival(ev::AbstractEvent)
    sim = environment(ev)
    time = now(sim)
    push!(times, time)
    push!(output, output[end]+1)
    if output[end] == 1
        service_delay = rand(service_distribution)
        @callback service(timeout(sim, service_delay))
    end
    next_arrival_delay = rand(interarrival_distribution)
    @callback arrival(timeout(sim, next_arrival_delay))
end

In [ ]:
function service(ev::AbstractEvent)
    sim = environment(ev)
    time = now(sim)
    push!(times, time)
    push!(output, output[end]-1)
    if output[end] > 0
        service_delay = rand(service_distribution)
        @callback service(timeout(sim, service_delay))
    end
end

In [ ]:
times = Float64[0.0]
output = Int[0]
sim = Simulation()
next_arrival_delay = rand(interarrival_distribution)
@callback arrival(timeout(sim, next_arrival_delay))
run(sim, 10.0)
println(times)
println(output)

- Two callback functions describe completely what happens during the execution of an event.
- For complicated systems (network of queues, clients with priorities, other scheduling methods) working with discrete events in this ways results in spaghetti code.
- Code reuse is very limited. A lot of very different application domains can be modeled in a similar way.

### Process-driven Discrete-event Simulation

- Events and their callbacks are abstracted and the simulation creator has only to program the logic of the system.
- A process function describes what a specific entity (also called agent) is doing.

In [ ]:
@resumable function packet_generator(sim::Simulation)
    line = Resource(sim, 1)
    while true
        next_arrival_delay = rand(interarrival_distribution)
        @yield timeout(sim, next_arrival_delay)
        @process packet(sim, line)
    end
end

@resumable function packet(sim::Simulation, line::Resource)
    time = now(sim)
    push!(times, time)
    push!(output, output[end]+1)
    @yield request(line)
    service_delay = rand(service_distribution)
    @yield timeout(sim, service_delay)
    time = now(sim)
    push!(times, time)
    push!(output, output[end]-1)
    @yield release(line)
end

In [ ]:
times = Float64[0.0]
output = Int[0]
sim = Simulation()
@process packet_generator(sim)
run(sim, 10.0)
println(times)
println(output)

## Plotting

In [ ]:
plot(times, output, line=:steppost, leg=false)
plot!(title = "MM1", xlabel = "Time", ylabel = "Number of clients in system")

## Monte Carlo Simulation and Statistical Processing

- Often we like to gather information about probabilites.
- We also want to know the variation of these probabilities between simulation runs.

In [ ]:
const RUNS = 30
const DURATION = 1000.0;

In [ ]:
Pₙ = Vector{Dict{Int, Float64}}()
for r in 1:RUNS
    push!(Pₙ, Dict{Int, Float64}())
    times = Float64[0.0]
    output = Int[0]
    sim = Simulation()
    @process packet_generator(sim)
    run(sim, DURATION)
    for (i,t) in enumerate(times[1:length(times)-1])
        duration = times[i+1] - t
        if output[i] ∈ keys(Pₙ[r])
            Pₙ[r][output[i]] = Pₙ[r][output[i]] + duration
        else
            Pₙ[r][output[i]] = duration
        end
    end
    tₑ = times[end]
    for nr_in_system in keys(Pₙ[r])
        Pₙ[r][nr_in_system] = Pₙ[r][nr_in_system] / tₑ
    end
    print("$(Pₙ[r][0]), ")
end
println()

In [ ]:
n = 8
arr = zeros(Float64, RUNS, n+1)
for v in 0:n
    for r in 1:RUNS
        if v ∈ keys(Pₙ[r])
            arr[r, v+1] = Pₙ[r][v]
        else
            arr[r, v+1] = 0
        end
    end
end
boxplot(reshape(collect(0:n), 1, n+1), arr, label=reshape(collect(0:n), 1, n+1))

## Hypothesistests

We can test easily whether that the data in a vector comes from a given distribution

In [ ]:
@resumable function packet(sim::Simulation, line::Resource)
    @yield request(line)
    service_delay = rand(service_distribution)
    @yield timeout(sim, service_delay)
    time = now(sim)
    push!(times, time)
    @yield release(line)
end

In [ ]:
times = Float64[]
sim = Simulation()
@process client_generator(sim)
run(sim, 100.0)
vec = diff(times)
ExactOneSampleKSTest(vec, interarrival_distribution)